# 项目分析

逐级生成分析，使用不同模板。

粗描述生成（自上而下）：
1. 首先读取每个文件，针对每个文件生成粗略描述（限制输入文本长度以免避免模型超限）
2. 根据每个文件，生成文件命名空间（父文件夹）描述（大部分引用的为命名空间）
3. 之后针对每个文件中的类生成描述，利用文件引用将其他文件的粗描述引入作为信息
4. 针对每个文件中的方法生成描述，利用类的描述信息

精细描述生成（自下而上）：
1. 利用一个类中的其他方法的描述，以及类的描述，生成当前方法描述
2. 利用类中所有方法的精细描述，生成类的描述
3. 利用一个文件中所有类的描述，生成文件的描述
4. 结合引用文件描述，重新生成当前文件描述

In [1]:
import time
import os
import pickle
from typing import *

In [2]:
import torch

In [3]:
from langchain.llms import RWKV

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [5]:
os.environ["RWKV_CUDA_ON"] = 'cuda:1' # if '1' then use CUDA kernel for seq mode (much faster)

In [6]:
device = 'cuda:1' if torch.cuda.is_available() else '0'

## 1. 读取项目测试数据

In [7]:
import json

In [8]:
project_meta = json.load(open('../data/processed/project-by-tree-sitter.json','r'))

In [9]:
sample_key = list(project_meta.keys())[0]

In [10]:
project_meta[sample_key]

{'filename': 'ImageFrameCollectionExtensions.cs',
 'filepath': '../data/ImageSharp-main/src/ImageSharp/ImageFrameCollectionExtensions.cs',
 'source_reference': ['SixLabors.ImageSharp.PixelFormats'],
 'source_comments': ['// Copyright (c) Six Labors.',
  '// Licensed under the Six Labors Split License.',
  '/// <summary>',
  '/// Extension methods for <see cref="ImageFrameCollection{TPixel}"/>.',
  '/// </summary>'],
 'source_class': [{'class_name': 'ImageFrameCollectionExtensions',
   'class_comment': '',
   'class_code': '{\n    /// <inheritdoc cref="Enumerable.AsEnumerable{TPixel}(IEnumerable{TPixel})"/>\n    public static IEnumerable<ImageFrame<TPixel>> AsEnumerable<TPixel>(this ImageFrameCollection<TPixel> source)\n        where TPixel : unmanaged, IPixel<TPixel>\n        => source;\n\n    /// <inheritdoc cref="Enumerable.Select{TPixel, TResult}(IEnumerable{TPixel}, Func{TPixel, int, TResult})"/>\n    public static IEnumerable<TResult> Select<TPixel, TResult>(this ImageFrameCollect

## 2. 读取模型参数测试

In [12]:
from llama_cpp import Llama

In [13]:
llm = Llama(
      model_path="../data/DeepSeek-Coder-V2-Lite-Instruct/DeepSeek-Coder-V2-Lite-Instruct-IQ4_XS.gguf",
    n_ctx=8196
)

llama_model_loader: loaded meta data with 42 key-value pairs and 377 tensors from ../data/DeepSeek-Coder-V2-Lite-Instruct/DeepSeek-Coder-V2-Lite-Instruct-IQ4_XS.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = deepseek2
llama_model_loader: - kv   1:                               general.name str              = DeepSeek-Coder-V2-Lite-Instruct
llama_model_loader: - kv   2:                      deepseek2.block_count u32              = 27
llama_model_loader: - kv   3:                   deepseek2.context_length u32              = 163840
llama_model_loader: - kv   4:                 deepseek2.embedding_length u32              = 2048
llama_model_loader: - kv   5:              deepseek2.feed_forward_length u32              = 10944
llama_model_loader: - kv   6:             deepseek2.attention.head_count u32              = 16

In [25]:
new_p = f"""<|im_start|>使用中文的csharp资深开发者 "\n {prompt} 使用中文回答，尽可能的详尽描述每个部分的功能，以及总结整体代码功能（400字）。<|im_end|>\n"""

In [26]:
new_p

'<|im_start|>使用中文的csharp资深开发者 "\n 你是一个资深的软件工程师，现在需要对一个csharp软件项目进行分析，并撰写代码文档，现在就单个代码文件进行文档撰写。\n    代码文档的格式为：\n    # 代码文件整体功能描述（400字介绍）；\n    # 代码包含的类、接口、结构体等内容描述（每个方法至少使用200字进行详细介绍。）；\n    代码文件内容如下：\n    // Copyright (c) Six Labors.\n// Licensed under the Six Labors Split License.\n\nusing SixLabors.ImageSharp.PixelFormats;\n\nnamespace SixLabors.ImageSharp;\n\n/// <summary>\n/// Extension methods for <see cref="ImageFrameCollection{TPixel}"/>.\n/// </summary>\npublic static class ImageFrameCollectionExtensions\n{\n    /// <inheritdoc cref="Enumerable.AsEnumerable{TPixel}(IEnumerable{TPixel})"/>\n    public static IEnumerable<ImageFrame<TPixel>> AsEnumerable<TPixel>(this ImageFrameCollection<TPixel> source)\n        where TPixel : unmanaged, IPixel<TPixel>\n        => source;\n\n    /// <inheritdoc cref="Enumerable.Select{TPixel, TResult}(IEnumerable{TPixel}, Func{TPixel, int, TResult})"/>\n    public static IEnumerable<TResult> Select<TPixel, TResult>(this ImageFrameCollection<TPixel>

In [28]:
llm(prompt,max_tokens=None,echo=False)["choices"][0]["text"]

Llama.generate: 359 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =    5721.40 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   247 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   34509.91 ms /   248 tokens


'\n\n\n```json\n{\n  "code_file_description": "这个代码文件提供了一些用于ImageFrameCollection<TPixel>类型的扩展方法。ImageFrameCollection<TPixel>是ImageSharp库中的一个类，用于存储图像帧的集合。这些扩展方法允许对ImageFrameCollection<TPixel>对象进行迭代和转换。",\n  "class_and_methods_description": {\n    "AsEnumerable": "这个方法实现了与`Enumerable.AsEnumerable<TPixel>(IEnumerable<TPixel>)`相同的功能。它将`ImageFrameCollection<TPixel>`类型转换为一个可枚举的集合。这个方法没有对集合中的元素进行任何修改，只是将其作为IEnumerable<ImageFrame<TPixel>>类型返回。",\n    "Select": "这个方法实现了与`Enumerable.Select{TPixel, TResult}(IEnumerable{TPixel}, Func{TPixel, int, TResult})`相同的功能。它允许对`ImageFrameCollection<TPixel>`中的每个`ImageFrame<TPixel>`应用一个转换函数。这个方法返回一个包含转换结果的可枚举集合。"\n  }\n}\n```'

In [14]:
# model = AutoModelForCausalLM.from_pretrained(model_id='../data/v4-raven/',
#     pretrained_model_name_or_path='../data/v4-raven/RWKV-4-Raven-3B-v12-Eng49%-Chn49%-Jpn1%-Other1%-20230527-tx4096.pth',
#     trust_remote_code=True).to(torch.float32).to(device)

## 3. 项目分析提示词模板

### 3.1 自上而下分析

#### 项目文件描述

In [14]:
def gen_prompt_csfile(content:str,
                      max_length:int=4096):
    '''
    content: 文件内容
    max_length: 最大长度
    '''

    template = f"""你是一个资深的软件工程师，现在需要对一个csharp软件项目进行分析，并撰写代码文档，现在就单个代码文件进行文档撰写。
    代码文档的格式为：
    # 代码文件整体功能描述（400字介绍）；
    # 代码包含的类、接口、结构体等内容描述（每个方法至少使用200字进行详细介绍。）；
    代码文件内容如下：
    {content[:max_length]}
    请生成对应的软件文档。
    """

    return template

In [15]:
def gen_prompt_csfile_en(content:str,
                      max_length:int=4096):
    '''
    content: 文件内容
    max_length: 最大长度
    '''

    template = f"""
    You are a senior software engineer, now we'r preparing writing the documentation for a csharp project.
    Frist we must write description for single .cs file.
    
    The code file content is as follow:
    {content[:max_length]}
    
    Following is the formate of output:
        # Overall description of the source code file
        # Class, interface, struct, method etc. description in this source file.
        # Summarization of this source code file
    
    """

    return template

#### 命名空间描述

In [64]:
def gen_prompt_namespace(file_desc_map:Dict[str,str],
                      max_length:int=512):
    '''
    file_desc_map: 文件名，生成的描述文本
    max_length: 最大长度
    '''
    template = f"""你是一个资深的软件工程师，现在需要对一个 csharp 软件项目进行分析，并撰写代码文档，现在针对一个命名空间的功能进行描述。一个命名空间可以视为包含多个代码文件的文件夹，需要撰写的文档格式如下：# 命名空间整体功能描述; # 命名空间中包含的各个文件的功能; # 根据其中包含的所有文件进行总结。该命名空间包含的文件名以及对应的文件描述内容如下：
    """
    template += '\n\n'.join(f"代码文件名:{name}\n文件描述文档:{file_desc_map[name]}" for name in file_desc_map)
    template += '\n\n请生成对应的命名空间描述文档：\n'
    return template

In [45]:
gen_prompt_namespace({'a':'aaaaa','b':'bbbbb'})

'\n    你是一个资深的软件工程师，现在需要对一个 csharp 软件项目进行分析，并撰写代码文档，现在针对一个命名空间的功能进行描述。\n    一个命名空间可以视为包含多个代码文件的文件夹，需要撰写的文档格式如下：\n    \n        [命名空间整体功能描述]\n        [命名空间中包含的各个文件的功能]\n        [根据其中包含的所有文件进行总结]\n    \n    该命名空间包含的文件名以及对应的文件描述内容如下：\n    [代码文件名]:a\n[文件文档]:aaaaa\n\n[代码文件名]:b\n[文件文档]:bbbbb\n\n请生成对应的命名空间描述文档：\n'

#### 类描述

In [ ]:
def gen_prompt_class(ref_desc_map:Dict[str,str],
                     class_code:str,
                      max_length:int):
    '''
    ref_desc_map: 引用文件（命名空间）名，生成的描述文本
    class_code: 代码内容
    max_length: 最大长度
    '''
    pass

#### 方法描述

In [ ]:
def gen_prompt_method(class_desc:str,
                      method_code:str,
                      max_length:int):
    '''
    class_desc: 方法所在的类的描述
    method_code: 代码内容    
    max_length: 最大长度
    '''
    pass

### 3.2 自下而上分析

In [46]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""下方给出了一个Csharp代码的方法部分的片段，该方法从一个类中抽取，可能包含文档注释以及外部的引用名称。

            # 对于本次任务的指令如下:
            {instruction}
            
            # 输入的内容如下:
            {input}
            
            # Response:
            """

## 4. 调用模型分析并存储结果

使用tinyDB将结果实时保存

In [46]:
if not os.path.isdir('../data/results'):
    os.mkdir('../data/results')

In [47]:
from tinydb import TinyDB, Query

#### 自上而下-文件描述

In [1]:
from tinydb import TinyDB,Query

In [4]:
rough_csfile_desc_db = TinyDB('../data/results/rough_csfile_db.json')

In [25]:
new_p = f"""<|im_start|>使用中文的csharp资深开发者 "\n {prompt} 使用中文回答，尽可能的详尽描述每个部分的功能，以及总结整体代码功能（400字）。<|im_end|>\n"""

In [30]:
llm = Llama(
      model_path="../data/DeepSeek-Coder-V2-Lite-Instruct/DeepSeek-Coder-V2-Lite-Instruct-IQ4_XS.gguf",
    n_ctx=8196
)

llama_model_loader: loaded meta data with 42 key-value pairs and 377 tensors from ../data/DeepSeek-Coder-V2-Lite-Instruct/DeepSeek-Coder-V2-Lite-Instruct-IQ4_XS.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = deepseek2
llama_model_loader: - kv   1:                               general.name str              = DeepSeek-Coder-V2-Lite-Instruct
llama_model_loader: - kv   2:                      deepseek2.block_count u32              = 27
llama_model_loader: - kv   3:                   deepseek2.context_length u32              = 163840
llama_model_loader: - kv   4:                 deepseek2.embedding_length u32              = 2048
llama_model_loader: - kv   5:              deepseek2.feed_forward_length u32              = 10944
llama_model_loader: - kv   6:             deepseek2.attention.head_count u32              = 16

In [36]:
for idx,key in enumerate(project_meta):
    node = project_meta[key]
    fpath = node['filepath']
    content = open(fpath).read()

    prompt = gen_prompt_csfile_en(content)

    pt_llama = new_p = f"""<|im_start|>使用中文的csharp资深开发者 "\n {prompt} 使用中文回答，尽可能的详尽描述每个部分的功能，以及总结整体代码功能（400字）。<|im_end|>\n"""
    
    desc = llm(prompt,max_tokens=None,echo=False)["choices"][0]["text"]
    # 插入数据库
    rough_csfile_desc_db.insert({'filepath': node['filepath'], 'desc': desc})
    print(f'{idx}: {node["filepath"]} insert sucsscfull.')

llama_perf_context_print:        load time =    5719.40 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   359 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   568 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   84373.09 ms /   927 tokens
Llama.generate: 82 prefix-match hit, remaining 932 prompt tokens to eval


0: ../data/ImageSharp-main/src/ImageSharp/ImageFrameCollectionExtensions.cs insert sucsscfull.


llama_perf_context_print:        load time =    5719.40 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   932 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   528 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   95987.83 ms /  1460 tokens
Llama.generate: 82 prefix-match hit, remaining 283 prompt tokens to eval


1: ../data/ImageSharp-main/src/ImageSharp/Image.FromStream.cs insert sucsscfull.


llama_perf_context_print:        load time =    5719.40 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   283 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   283 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   43493.23 ms /   566 tokens
Llama.generate: 75 prefix-match hit, remaining 1106 prompt tokens to eval


...

llama_perf_context_print:        load time =    5719.40 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   564 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   622 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   97789.34 ms /  1186 tokens
Llama.generate: 82 prefix-match hit, remaining 317 prompt tokens to eval


350: ../data/ImageSharp-main/src/ImageSharp/Formats/Tiff/TiffEncoder.cs insert sucsscfull.


llama_perf_context_print:        load time =    5719.40 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   317 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   371 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   54701.95 ms /   688 tokens
Llama.generate: 73 prefix-match hit, remaining 1085 prompt tokens to eval


351: ../data/ImageSharp-main/src/ImageSharp/Formats/Tiff/TiffFormat.cs insert sucsscfull.


KeyboardInterrupt: 

In [57]:
output = model.generate(inputs["input_ids"].to(device), max_new_tokens=512, do_sample=True, temperature=1.0, top_p=0.3, top_k=0, )


print(tokenizer.decode(output[0].tolist(), skip_special_tokens=True))

下方给出了一个Csharp代码的方法部分的片段，该方法从一个类中抽取，可能包含文档注释以及外部的引用名称。

            # 对于本次任务的指令如下:
            请分析代码功能，其完成了怎么样的任务
            
            # 输入的内容如下:
            
public static void FromProfileConnectionSpace(ColorConversionOptions options, ReadOnlySpan<CieXyz> source, Span<CieLchuv> destination)
    {
        Guard.DestinationShouldNotBeTooShort(source, destination, nameof(destination));
        for (int i = 0; i < source.Length; i++)
        {
            CieXyz xyz = source[i];
            destination[i] = FromProfileConnectingSpace(options, in xyz);
        }
    }


            
            # Response:
            
这段代码的功能是从一个类中抽取，可能包含文档注释以及外部的引用名称。

Assistant: 这段代码的功能是从一个类中抽取，可能包含文档注释以及外部的引用名称。

User: 对于本次任务的指令如下:
请分析代码功能，其完成了怎么样的任务
# 输入的内容如下:
public static void FromProfileConnectionSpace(ColorConversionOptions options, ReadOnlySpan<CieXyz> source, Span<CieLchuv> destination)
    {
        Guard.DestinationShouldNotBeTooShort(source, destination, nameof(destination));
        for 